In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：使用导出数据集进行在线预测的自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库训练和部署自定义图像分类模型用于在线预测，采用导出的`Dataset`资源。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Flowers数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。您在本教程中将使用的数据集版本存储在公共云存储桶中。训练的模型可以预测图像属于五种花中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 目标

在本教程中，您将学习如何使用在Docker容器中的Python脚本中导出的`Dataset`资源来创建一个自定义模型，使用Vertex客户端库，并对部署的模型进行预测。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建模型。

执行的步骤包括：

- 创建Vertex`Dataset`资源。
- 导出`Dataset`资源的清单。
- 为训练模型创建Vertex自定义作业。
- 导入导出的数据集清单。
- 训练模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex`Model`资源。
- 部署`Model`资源到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

成本

本教程使用了 Google Cloud（GCP）的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 *google-cloud-storage* 库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本核心，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有GPU运行时选项，请确保在此笔记本中使用GPU运行时。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的谷歌云项目

**以下步骤是必需的，无论您使用的是哪种笔记本环境。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保
Cloud SDK 使用正确的项目执行此笔记本中的所有命令。

**注意**：Jupyter 会将以 `!` 为前缀的行视为 shell 命令，它会将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改`REGION`变量，该变量用于笔记本其余部分的操作。 以下是 Vertex 支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来进行 Vertex 的训练。并非所有地区都支持所有 Vertex 服务。有关每个地区的最新支持，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请执行以下步骤：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，然后选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐号密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

当您使用Vertex客户端库提交自定义训练作业时，您需要将包含您的训练代码的Python软件包上传到一个云存储存储桶中。Vertex会从这个软件包中运行代码。在本教程中，Vertex还会将作业产生的训练模型保存在同一个存储桶中。然后您可以基于此输出创建一个`Endpoint`资源，以提供在线预测。

请在下面输入您的云存储存储桶的名称。存储桶的名称必须在所有谷歌云项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常数

### 导入库并定义常数

#### 导入 Vertex 客户端库

将 Vertex 客户端库导入我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、管道和终端服务端点。
- `PARENT`：数据集、模型、作业、管道和终端资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

标签常量

设置`Dataset`资源的独特常量标签

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"

硬件加速器

设置用于训练和预测的硬件加速器（例如，GPU）。

设置`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`变量以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个包含4个Nvidia Telsa K80 GPUs的GPU容器映像分配给每个VM，您应指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以在CPU上运行容器映像。

*注意*：TF 2.3之前的GPU支持版本将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复，这是由生成在serving函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

- TensorFlow 1.15
  - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
  - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
  - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
  - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
  - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
  - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

最新列表，请参阅[用于训练的预建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
  - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
  - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
  - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
  - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
  - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

最新列表，请参阅[用于预测的预建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存。
     - `n1-highcpu`：每个虚拟CPU0.9GB内存。
 - `虚拟CPU个数`：\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下不支持用于训练:*

 - `standard`：2个虚拟CPU
 - `highcpu`：2、4和8个虚拟CPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好为花朵创建自己的定制模型并进行训练了。

## 设置客户端

Vertex客户端库以客户端/服务器模型运行。在您的一侧（Python脚本）上，您将创建一个客户端，从Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程的不同步骤使用不同的客户端。所以，请提前为它们全部设置。

- 用于`Dataset`资源的数据集服务。
- 用于`Model`资源的模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户已经准备好，训练模型的第一步是创建一个托管的数据集实例，然后将您的标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
   - `display_name`：您选择的可读名称。
   - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法 `create_dataset`，具有以下参数：
   - `parent`：您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
   - `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用的方式。虽然这一步通常很快，但在您首次在项目中使用时，由于需要进行配置，可能会有较长的延迟。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源）或取消操作，通过调用操作方法：

| 方法         | 描述          |
| ------------ | ------------ |
| result()     | 等待操作完成并返回 JSON 格式的结果对象。 |
| running()    | 返回操作是否仍在运行的 True/False 值。 |
| done()       | 返回操作是否已经完成的 True/False 值。 |
| canceled()   | 返回操作是否已取消的 True/False 值。 |
| cancel()     | 取消操作（可能需要多达 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("flowers-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

数据准备

Vertex中的“数据集”资源用于图像，对您的数据有一些要求：

- 图像必须存储在Cloud Storage存储桶中。
- 每个图像文件必须是图像格式（PNG、JPEG、BMP等）。
- 必须在您的Cloud Storage存储桶中存储一个包含每个图像路径和标签的索引文件。
- 索引文件必须是CSV或JSONL格式。

CSV索引文件的要求如下：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

您将使用存储在公共云存储桶中的 Flowers 数据集的一个版本，使用一个 CSV 索引文件。

从快速查看数据开始。您可以通过计算 CSV 索引文件中的行数（`wc -l`）来计算示例的数量，然后浏览前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并传入以下参数：
  - `name`：您对 `Dataset` 资源命名的人类可读名称（例如，flowers）。
  - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，具有以下键/值条目：
  - `gcs_sources`：URI 列表，指向一个或多个索引文件的路径。
  - `import_schema_uri`：标识标记类型的模式。

`import_data()` 方法将返回一个长时间运行的 `operation` 对象。这将需要几分钟来完成。如果您正在进行实时教程，在这个时候可以提问或休息片刻。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

### 导出数据集索引

接下来，您将把数据集索引导出到一个 JSONL 文件中，然后您的自定义训练任务将使用该文件获取数据和相应的标签，用于训练您的 Flowers 模型。使用这个辅助函数 `export_data` 来导出数据集索引。该函数执行以下操作：

- 使用数据集客户端。
- 调用客户端方法 `export_data`，传入以下参数：
  - `name`：您给数据集的可读名称（例如 flowers）。
  - `export_config`：导出配置。
- `export_config`：一个包含字典的 Python 列表，其中包含以下键值对：
  - `gcs_destination`：要将 JSONL 数据集索引文件写入的 Cloud Storage 存储桶。

`export_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟的时间才能完成。当导出完成时，辅助函数将返回长时间运行的操作和操作的结果。

In [ ]:
EXPORT_FILE = BUCKET_NAME + "/export"


def export_data(dataset_id, gcs_dest):
    config = {"gcs_destination": {"output_uri_prefix": gcs_dest}}
    start_time = time.time()
    try:
        operation = clients["dataset"].export_data(
            name=dataset_id, export_config=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation, result
    except Exception as e:
        print("exception:", e)
        return None, None


_, result = export_data(dataset_id, EXPORT_FILE)

### 快速查看您的导出数据集索引文件

现在让我们快速浏览导出数据集索引文件的内容。当`export_data()`完成时，可以从长时间运行操作的`result()`方法中获取响应对象。响应对象包含以下属性：

- `exported_files`: 导出数据集索引文件的路径列表，在这种情况下将为一个文件。

您将获得导出数据集索引文件的路径(`result.exported_files[0]`)，然后显示文件中的前十个JSON对象--即数据项。

每个数据项的JSONL格式如下：

    { "imageGcsUri": 图像路径, "classificationAnnotation": { "displayName": 标签 } }

In [ ]:
jsonl_index = result.exported_files[0]

! gsutil cat $jsonl_index | head

#### 读取索引文件

您需要向自定义训练的 Python 脚本中添加代码，以读取导出的数据集索引，从而能够为自定义训练模型生成训练批次。

以下是如何读取导出的数据集索引文件的示例：

1. 使用 Tensorflow 的 Cloud Storage 文件方法来打开文件（`tf.io.gfile.GFile()`），并读取所有行（`f.readlines()`），每一行都代表一个 JSONL 对象的数据项。

2. 针对文件中的每一行，将该行转换为一个 JSON 对象（`json.loads()`）。

3. 提取图像的路径（`['imageGcsUri']`）和标签（`['classificationAnnotation']['displayName']`）。

In [ ]:
import json

import tensorflow as tf

with tf.io.gfile.GFile(jsonl_index, "r") as f:
    export_data_items = f.readlines()

for _ in range(10):
    j = json.loads(export_data_items[_])
    print(j["imageGcsUri"], j["classificationAnnotation"]["displayName"])

#### 创建TFRecords

接下来，我们需要创建一个数据馈送机制，从数据集索引文件中向你训练的模型提供数据。关于如何构建数据馈送器有很多选择。我们将在这里介绍两种选项，都是使用TFRecords:

1. 将图像数据存储为原始未压缩的图像数据（每个像素1字节）。
2. 将图像数据存储为预处理过的数据--机器学习准备好的--（每个像素4字节）。

这两种方法展示了磁盘存储和计算时间之间的权衡。在这两种情况下，我们对图像数据进行一次预处理，将数据缓存成一种形式，将加快模型的训练时间。但是，通过缓存，我们既使用了磁盘空间，又增加了从磁盘到计算设备（如CPU、GPU、TPU）的I/O流量。

在原始未压缩格式中，你最大程度地减少了磁盘上的大小和I/O流量用于缓存数据，但每个epoch，图像数据的预处理都需要重复执行。在预处理格式中，通过一次预处理和缓存预处理数据来最小化计算时间--即准备好机器学习。与Float32训练时，磁盘上的数据量将是四倍大小，同时你将增加相同数量的磁盘空间和从磁盘到计算引擎的I/O流量。

辅助函数`TFExampleImageUncompressed`和`TFExampleImagePreprocessed`都接受以下参数：

- `path`：图像文件的Cloud Storage路径。
- `label`：图像文件的相应标签。
- `shape`：要调整图像大小的(H,W)输入形状。如果为`None`，则不执行调整大小。

辅助函数`TFExampleImagePreprocessed`有一个额外的参数：

- `dtype`：像素数据被归一化后的浮点表示。默认设置为32位浮点（np.float32）。如果你使用NVIDIA GPU或TPU，可以将`dtype = np.float16`来训练16位浮点。在不影响准确性或迭代次数的情况下，使用16位浮点训练有两个好处：

1. 每次矩阵乘法操作比32位相当的操作快4倍--尽管模型权重也需要以16位存储。
2. 磁盘空间和I/O带宽减少了1/2。

让我们更深入地看一下用于创建`TFRecord`训练数据的函数。首先，`TFRecord`是训练数据的序列化二进制编码。作为编码，需要为字段的编码指定模式，然后在向模型馈送训练数据时使用后续解码。

模式被定义为每个训练数据中的`tf.train.Example`实例。每个`tf.train.Example`实例由一系列字段组成，每个字段定义为键/值对。在我们的辅助函数中，键条目为：

- `image`：编码的原始图像数据。
- `label`：分配给图像的标签。
- `shape`：解码时图像的形状。

每个键/值对的值是一个`tf.train.Feature`实例，其中：

- `bytes_list`：要编码的数据是字节字符串。
- `int64_list`：要编码的数据是一个或多个整数值的数组。
- `float_list`：要编码的数据是一个或多个浮点值的数组。

In [ ]:
import numpy as np


def TFExampleImageUncompressed(path, label, shape=None):
    """ The uncompressed version of the image """

    # read in (and uncompress) the image
    with tf.io.gfile.GFile(path, "rb") as f:
        data = f.read()
    image = tf.io.decode_image(data)

    if shape:
        image = tf.image.resize(image, shape)
    image = image.numpy()
    shape = image.shape

    # make the record
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image": tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image.tostring()])
                ),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                "shape": tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[shape[0], shape[1], shape[2]])
                ),
            }
        )
    )


def TFExampleImagePreprocessed(path, label, shape=None, dtype=np.float32):
    """ The normalized version of the image """

    # read in (uncompress) the image and normalize the pixel data
    image = (cv2.imread(path) / 255.0).astype(dtype)

    if shape:
        image = tf.image.resize(image, shape)
    image = image.numpy()
    shape = image.shape

    # make the record
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image": tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image.tostring()])
                ),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                "shape": tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[shape[0], shape[1], shape[2]])
                ),
            }
        )
    )

#### 将训练数据写入TFRecord文件

接下来，您将为导出的数据集索引中指定的所有训练数据创建一个单个TFRecord：

- 通过将变量`CACHE`设置为`TFExampleImageUncompressed`或`TFExampleImagePreprocessed`来指定缓存方法。
- 使用`cls2label`将数据集中的类名转换为整数标签。
- 从导出的数据集索引文件中读取数据项列表 - `tf.io.gfile.GFile(jsonl_index, 'r')`。
- 设置用于存储缓存TFRecord文件的Cloud Storage位置 - `GCS_TFRECORD_URI`。
- 使用`tf.io.TFRecordWriter(gcs_tfrecord_uri)`为导出的数据集索引中的每个数据项生成缓存数据。
 - 提取Cloud Storage路径和类名 - `json.loads(data_item)`
 - 将类名转换为整数标签 - `label = cls2label[cls]`
 - 编码数据项 - `example = CACHE(image, label)`
 - 将编码的数据项写入TFRecord文件 - `writer.write(example.SerializeToString())`

这可能需要大约20分钟。

In [ ]:
# Select TFRecord method of encoding
CACHE = TFExampleImageUncompressed  # [ TFExampleImageUncompressed, TFExampleImagePreprocessed]

# Map labels to class names
cls2label = {"daisy": 0, "dandelion": 1, "roses": 2, "sunflowers": 3, "tulips": 4}

# Read in each example from exported dataset index
with tf.io.gfile.GFile(jsonl_index, "r") as f:
    data = f.readlines()

# The path to the TFRecord cached file.
GCS_TFRECORD_URI = BUCKET_NAME + "/flowers.tfrecord"

# Create the TFRecord cached file
with tf.io.TFRecordWriter(GCS_TFRECORD_URI) as writer:
    n = 0
    for data_item in data:
        j = json.loads(data_item)
        image = j["imageGcsUri"]
        cls = j["classificationAnnotation"]["displayName"]
        label = cls2label[cls]
        example = CACHE(image, label, shape=(128, 128))
        writer.write(example.SerializeToString())
        n += 1
        if n % 10 == 0:
            print(n, image)

listing = ! gsutil ls -la $GCS_TFRECORD_URI
print("TFRecord File", listing)

## 训练模型

有两种方法可以使用容器映像来训练自定义模型：

- **使用 Google Cloud 的预构建容器**。如果您使用预构建容器，还需要指定一个要安装到容器映像中的 Python 包。这个 Python 包包含用于训练自定义模型的代码。

- **使用您自己的定制容器映像**。如果您使用自己的容器，容器需要包含用于训练自定义模型的代码。

## 准备您的定制工作规范

现在您的客户准备好了，您的第一步是为您的定制培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规范以及使用的数量（单个或分布式）
- `python_package_spec`：要与预构建容器一起安装的Python软件包规范。

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这告诉 Vertex 要为训练提供什么类型的机器实例。
- `machine_type`：要提供的 GCP 实例类型，例如 n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则使用 GPU；否则将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这告诉 Vertex 需要为训练中的每台机器实例提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或 Standard 中的一个。SSD 更快，Standard 更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

PARAM_FILE = BUCKET_NAME + "/params.txt"

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_flowers.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装自定义工作规范的完整描述：

- `display_name`：您为此自定义工作指定的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务在哪里保存模型工件的云存储位置（当变量 `DIRECT = False`时）。 然后，服务将该位置传递给训练脚本作为环境变量 `AIP_MODEL_DIR`，路径将采用以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为定制培训工作组装Python包。解压缩后，该包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的指令。

文件`trainer/task.py`是执行定制培训工作的Python脚本。*注意*，当我们在工作组规范中提到它时，我们将目录斜杠替换为点（`trainer.task`）并删除文件后缀（`.py`）。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Flowers image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py文件内容

在下一个单元格中，您可以编写训练脚本task.py的内容。我们不会详细讨论，只是让您浏览一下。总之：

- 从命令行中获取要保存模型产物的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TFRecords加载Flowers数据集（`args.training-data`）。
- 从TFRecord创建一个tf.data.Dataset生成器。
- 使用TF.Keras模型API构建一个简单的ConvNet模型。
- 编译模型（`compile()`）。
- 使用`args.distribute`为多个工作进程设置训练分发策略。
- 使用根据参数`args.epochs`和`args.steps`设置的epochs和steps来训练模型（`fit()`）。
- 将训练后的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Flowers

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--training-data', dest='tfrecord_uri')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():
    dataset = tf.data.TFRecordDataset(args.tfrecord_uri)

    feature_description = {
        'image': tf.io.FixedLenFeature([],  tf.string),
        'label': tf.io.FixedLenFeature([],  tf.int64),
        'shape': tf.io.FixedLenFeature([3], tf.int64),
    }


    def _parse_function(proto):
        ''' parse the next serialized tf.train.Example using the feature description '''
        example = tf.io.parse_single_example(proto, feature_description)
        image = tf.io.decode_raw(example['image'], tf.int32)
        shape = tf.cast(example['shape'], tf.int32)
        label = tf.cast(example['label'], tf.int32)
        image.set_shape([128 * 128 * 3])
        image = tf.reshape(image, (128, 128, 3))
        image = ( tf.cast(image, tf.float32) / 255.0)

        return image, label


    return dataset.map(_parse_function).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(128, 128, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(128, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(5, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_flowers.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

辅助函数调用作业客户服务的`create_custom_job`方法，使用以下参数：

- `parent`：Vertex资源`Dataset`，`Model`和`Endpoint`的位置路径。
- `custom_job`：自定义作业的规范。

您将显示`response`对象返回的一些字段，其中最感兴趣的两个是：

`response.name`：为此自定义训练作业分配的Vertex完全限定标识符。您可以在后续步骤中使用此标识符。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，带有以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

如果您记得，在调用 `create_custom_job` 方法时，您可以在 `response.name` 字段中获得自定义作业的 Vertex 完全限定标识符，并将该标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

#部署

训练上述模型可能需要20分钟以上的时间。

一旦您的模型训练完成，您可以通过将“结束时间”减去“开始时间”来计算训练模型所需的实际时间。对于你的模型，我们需要知道保存模型的位置，Python脚本会将其保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 中的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将使用 `tf.keras.datasets` 中的 `load_data()` 方法加载 Flowers 测试（留置）数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个元素是测试数据。每个元素还是一个包含两个元素的元组：图像数据和相应的标签。

您不需要训练数据，因此我们将其加载为 `(_,_)`。

在运行数据进行评估之前，您需要对其进行预处理：

x_test:
1.通过将每个像素除以255来标准化（重新缩放）像素数据。这将用0到1之间的32位浮点数替换每个单字节整数像素。

y_test:<br/>
2.标签当前是标量（稀疏）。如果查看`trainer/task.py`脚本中的`compile()`步骤，您将发现它是为稀疏标签编译的。因此我们不需要再做任何事情。

## 评估模型

现在让我们来评估模型的表现。

### 加载评估数据

您将加载来自Flowers数据集末尾的一些样本数据，即最后10个项目。然后我们将对数据项进行预处理，形成：

- `x_test`：内存中的预处理图像数据。
- `y_test`：对应的标签。

In [ ]:
x_test = []
y_test = []

data_items = export_data_items[-10:]
for data_item in data_items:
    data_item = json.loads(data_item)
    print("FILE", data_item["imageGcsUri"])
    with tf.io.gfile.GFile(data_item["imageGcsUri"], "rb") as f:
        data = f.read()
    image = tf.io.decode_image(data)
    image = tf.image.resize(image, (128, 128))
    image = (image.numpy() / 255.0).astype(np.float32)
    cls = data_item["classificationAnnotation"]["displayName"]
    label = cls2label[cls]
    x_test.append(image)
    y_test.append(label)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

### 进行模型评估

现在评估定制工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型以供服务

接下来，您将从自定义作业将您的TF.Keras模型上传到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个serving函数，将数据转换为模型所需的格式。如果您向Vertex发送编码数据，则您的serving函数会确保在将数据作为输入传递给您的模型之前，在模型服务器上对数据进行解码。

### serving函数如何工作

当您向在线预测服务器发送请求时，请求将由HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取预测请求。提取的预测请求被转发到serving函数。对于Google预构建的预测容器，请求内容以`tf.string`形式传递给serving函数。

serving函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型（动态图）的输入形状和数据类型。
  - 执行与在训练底层模型期间进行的数据预处理相同的处理--例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式--例如，压缩输出。
  - 为接收应用程序打包输出--例如，添加标题，创建JSON对象等。

预处理和后处理函数都会转换为与模型融合的静态图。底层模型的输出传递给后处理函数。后处理函数将转换/打包后的输出返回给HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建serving函数时，您需要考虑的一点是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果您这样做，那么在编译serving函数时将会出现错误，该错误将指示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩的（例如 JPEG）图像字节编码为 base 64 格式 -- 这将使内容在通过网络传输二进制数据时免受修改。由于部署的模型期望输入数据为原始（未压缩）字节，因此您需要确保将 base 64 编码数据转换回原始字节，然后将其作为输入传递给部署的模型。

为了解决这个问题，请定义一个服务功能（`serving_fn`），并将其附加到模型作为预处理步骤。添加一个 `@tf.function` 装饰器，以便服务功能与底层模型融合（而不是在 CPU 上游）。

当您发送预测或解释请求时，请求的内容将被 base 64 解码为一个 Tensorflow 字符串 (`tf.string`)，然后传递给服务功能（`serving_fn`）。服务功能将 `tf.string` 预处理为原始（未压缩）的 numpy 字节（`preprocess_fn`）以匹配模型的输入要求：
- `io.decode_jpeg`- 解压缩 JPG 图像，作为一个具有三个通道（RGB）的 Tensorflow 张量返回。
- `image.convert_image_dtype` - 将整数像素值转换为浮点 32，并重新缩放像素数据在 0 到 1 之间。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

此时，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(128, 128))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 128, 128, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

## 获取serving函数的签名

通过重新加载模型，并对每一层查询对应的签名，您可以获取模型输入和输出层的签名。

为了您的目的，您需要获取serving函数的签名。为什么呢？因为当我们将数据作为HTTP请求数据包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将执行从base64到numpy数组的转换。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称 -- 这会在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

将模型上传

使用这个辅助函数`upload_model`来上传您的模型，该模型以SavedModel格式存储，并将其上传到`Model`服务，该服务将为您的模型实例化一个Vertex`Model`资源实例。一旦您完成了这一步，您可以像任何其他Vertex`Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

该辅助函数使用以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel艺术品的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了它。

该辅助函数调用`Model`客户端服务的方法`upload_model`，它接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们深入了解一下Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex`Dataset`资源的情况下构建的，您将其留空(`''`)。
- `artificat_uri`：模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex模型资源会返回一个长时间运行的操作，因为这可能需要一些时间。您可以调用response.result()，这是一个同步调用，并且在Vertex模型资源准备就绪时将返回。

辅助函数返回相应的Vertex完全限定的标识符以获取相应的Vertex模型实例`upload_model_response.model`。您将保存该标识符以供后续步骤使用，保存在变量`model_to_deploy_id`中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "flowers-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取`Model`资源信息

现在让我们只获取您的模型的模型信息。使用这个辅助函数`get_model`，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用了Vertex `Model`客户端服务的`get_model`方法，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署已训练的Vertex自定义`Model`资源。这需要两个步骤：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建`Endpoint`资源

使用这个辅助函数`create_endpoint`创建一个用于部署模型以进行预测服务的终端节点，需要以下参数：

- `display_name`：`Endpoint`资源的可读性名称。

该辅助函数使用终端节点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的可读性名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为服务提供`Endpoint`资源。您调用`response.result()`，这是一个同步调用，当终端节点资源准备就绪时会返回。辅助函数返回`Endpoint`资源的Vertex完全合格标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "flowers_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“ 端点” 资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩缩容

您有几种选择可以对计算实例进行扩缩容，以处理您的在线预测请求：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩缩容：在线预测请求在您手动指定的固定数量的计算实例之间分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，固定数量的计算实例将被准备，并在线预测请求在它们之间均匀分配。

- 自动扩缩容：在线预测请求在可扩展数量的计算实例之间分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时要准备和取消准备的实例数量，并将最大（`MAX_NODES`）数量设置为根据负载条件要准备实例的数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将从训练管道上传（部署）的 `Model` 资源部署到您为提供预测创建的 `Endpoint` 资源，使用以下参数：

- `model`：要上传的模型的 Vertex 完全合格的模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：部署模型的 Vertex 完全合格的终端标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全合格的 `Endpoint` 资源标识符。
- `deployed_model`：用于部署模型的要求规范。
- `traffic_split`：传送到此模型的终点的流量百分比，指定为一个或多个键/值对的字典。
  - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是上传的这个模型，100 表示 100% 的流量。
  - 如果终端上有现有的模型，并且流量将被分配给它们，则使用 `model_id` 来指定为 **{ "0": 百分比，model_id: 百分比, ... }**，其中 `model_id` 是部署端点的现有模型的模型标识符。这些百分比必须加起来为 100。

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个具有最少必需字段的 Python 字典：

- `model`：要部署的（上传）模型的 Vertex 完全合格的模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这会禁用容器事件的记录，例如执行失败（默认情况下容器日志记录已启用）。容器日志记录通常在调试部署时启用，然后在生产部署时禁用。
- `dedicated_resources`：这是为提供预测请求而缩放的计算实例（副本）的数量。
  - `machine_spec`：要提供的计算实例。使用先前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`：初始要提供的计算实例数，您先前设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的计算实例的最大数量，您先前设置为变量 `MAX_NODES`。

#### 流量分割

现在让我们更深入地了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。刚开始可能有点困惑。让我解释一下，您可以将多个模型实例部署到一个终端，然后设置每个实例接收多少（百分比）的流量。

为什么要这样做？也许您已经在生产环境中部署了以前的版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估结果，但在将其部署到生产环境之前，您并不能确定它是否真的更好。因此，在流量分割的情况下，您可能希望将 v2 部署到与 v1 相同的终端，但只分配给它说 10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其表现 -- 直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，该方法将阻塞直到模型被部署。如果这是第一次将模型部署到终端，则可能需要额外几分钟来完成资源的分配。

In [ ]:
DEPLOYED_NAME = "flowers_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出在线预测请求

现在对您部署的模型进行在线预测。

In [ ]:
# Last data item in exported dataset index
data_items = export_data_items[-1:]
data_item = json.loads(data_items[0])
image_path = data_item["imageGcsUri"]
print("IMAGE PATH", image_path)

准备请求内容
您将发送图像作为压缩的JPG图像，而不是原始未压缩的字节：

- `cv2.imread`：将压缩的JPG图像读取回内存中作为原始字节。
- `tf.image.resize`：将图像调整大小为模型的输入形状--(128, 128, 3)。
- cv2.imwrite：将调整大小后的图像写回磁盘。
- `base64.b64encode`：读取调整大小后的压缩图像并编码为一个base 64编码的字符串。

In [ ]:
! gsutil cp $image_path tmp.jpg

import base64

import cv2

test_image = cv2.imread("tmp.jpg", cv2.IMREAD_COLOR)
print("before:", test_image.shape)
test_image = cv2.resize(test_image, (128, 128))
print("after:", test_image.shape)
cv2.imwrite("tmp.jpg", test_image.astype(np.uint8))

# bytes = tf.io.read_file('tmp.jpg')
with open("tmp.jpg", "rb") as f:
    bytes = f.read()
b64str = base64.b64encode(bytes).decode("utf-8")

### 发送预测请求

好的，现在你有了一个测试图片。使用这个辅助函数 `predict_image`，它接受以下参数：

- `image`: 作为 numpy 数组的测试图片数据。
- `endpoint`: Vertex 完全限定的标识符，指的是部署了 `Model` 资源的 `Endpoint` 资源。
- `parameters_dict`: 用于服务的额外参数。

此函数调用预测客户端服务的 `predict` 方法，并传递以下参数：

- `endpoint`: Vertex 完全限定的标识符，指的是部署了 `Model` 资源的 `Endpoint` 资源。
- `instances`: 要预测的一系列实例（编码后的图片）。
- `parameters`: 用于服务的额外参数。

为了将图片数据传递给预测服务，在上一步你将字节编码为 base64 格式 -- 这样可以保证在通过网络传输二进制数据时内容不会被修改。你需要告诉服务二进制代码你的模型部署在哪里，并且内容已经被 base64 编码，这样服务二进制代码在另一端会进行解码。

预测请求中的每个实例都是一个字典条目，格式如下：

                        {serving_input: {'b64': content}}

- `input_name`: 底层模型的输入层的名称。
- `'b64'`: 一个表示内容为 base64 编码的键。
- `content`: 压缩后的 JPG 图像字节，以 base64 编码的字符串形式。

由于 `predict()` 服务可以接受多个图片（实例），你将把你的单个图片作为一个图片列表发送。最后一步是将实例列表打包进 Google 的 protobuf 格式 -- 这就是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，其中列表中的每个元素对应请求中的每个图片。你会在每个预测输出中看到以下内容：

- `predictions`: 表示对每个类别的预测置信度，介于 0 到 1 之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

解除部署`Model`资源

现在从服务的`Endpoint`资源中解除部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当将`Model`资源部署到服务端点时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的`undeploy_model`方法，使用以下参数：

- `deployed_model_id`：当`Model`资源被部署时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上的其余部署模型之间分配流量。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将`traffic_split`设置为空对象{}来保持空白。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有 GCP 资源，您可以[删除用于教程的 GCP 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME